In [1]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson
from glm import GLM

In [2]:
N = 100000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [4]:
model.coef_

array([ 0.99996915, -1.99429859,  1.00387664])

## Linear Model with Sample Weights

In [14]:
sample_weights = np.random.uniform(0, 2, size=N)

In [15]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [16]:
model.coef_

array([ 1.0040524 , -1.99539289,  0.99870377])

## Logistic Model

In [5]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [6]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [7]:
model.coef_

array([ 0.98446972, -1.98823498,  1.0273801 ])

## Poission Model

In [8]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [9]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [10]:
model.coef_

array([ 1.01138576, -2.01161029,  0.98791408])

## Poisson with Exposures

In [11]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [12]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [13]:
model.coef_

array([ 0.99736751, -1.99991392,  1.00231158])